In [7]:
import pandas as pd
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)


In [8]:
# Check library version
!pip list | grep -E 'transformers|torch|Keras'

Keras                2.3.1     
Keras-Applications   1.0.8     
Keras-Preprocessing  1.1.0     
torch                1.4.0     
transformers         2.5.1     
You are using pip version 19.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [10]:
df_data = pd.read_csv('../data/text_classification_dataset.csv',sep=",",encoding="utf-8",names=['labels','texts'])
df_data.head()

,labels,texts
0,0,"god is great , the movie's not ."
1,0,. . . the whole thing succeeded only in making...
2,1,"light the candles , bring out the cake and don..."
3,1,"the story may not be new , but australian dire..."
4,1,you live the mood rather than savour the story .


## Pre-processing

In [11]:
# Get sentence data
sentences = df_data.texts.to_list()
sentences[0]

"god is great , the movie's not ."

In [12]:
# Get tag labels data
labels = df_data.labels.to_list()
print(labels[0])

0


In [13]:
# make tag name into index for training

# Set a dict for mapping id to tag name
#tag2idx = {t: i for i, t in enumerate(tags_vals)}

# Recommend to set it by manual define, good for reusing
# 0:negative, 1: positive
tag2idx={'0': 0,
 '1': 1}

In [14]:
tag2idx


{'0': 0, '1': 1}

In [15]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}

In [16]:
tag2name

{0: '0', 1: '1'}

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [18]:
n_gpu

0